
智能指针 boost::shared_ptr<>
共享数组
    boost::shared_array<>
    #include <boost/shared_array.hpp>
    声明：
        boost::shared_array<int> i1(new int[2]);
        boost::shared_array<int> i2(i1);
        i1[0]  = 1;
        >>>i2[0] =1

弱指针
    boost::weak_ptr<> 和shared_ptr一起使用
    weak_ptr通过shared_ptr来初始化 初始化后 只提供一个::lock()方法 该方法返回shared_ptr<>
    初始化弱指针的共享指针共享所有权

    使用弱指针的情况：
        函数需要一个由共享指针所管理的对象
        这个对象的生存期不依赖于这个函数
    
    (用的很少)

作用域指针
    boost::scoped_ptr
    不能传递它所包含的对象的所有权到另一个作用域指针

介入式指针  
    boost::intrusive_ptr<> 需要自己做记录
    对于COM（组件对象模型）有用
    COM对象需要记录当前有多少指针引用着它

指针容器
    boost::ptr_vector<> 管理动态分配的对象
    独占它所包含的对象，因此容器之外的共享指针不能共享所有权
    与vector<boost::shared_ptr<>> 相反
    #include <boost/ptr_container/ptr_vector.hpp>

函数对象

boost::bind:
    将这些函数与几乎不限数量的参数 一起使用 得到指定签名的函数
    #include <algorithm>

std::for_each():
    第三个参数是一个仅接受 一个参数的 函数或函数对象
    #include <algorithm>

for_each的例子：
    void print(int i){//打印i}
    int main(){
        vector<int> v;
        v.push_back(1);
        v.push_back(2);
        v.push_back(3);

        std::for_each(v.begin(),v.end(),print);

    }
    for_each()第三个参数必须是一个函数 这个函数只有一个参数

std::for_each + std::bindlst 使 for_each的第三个参数 这个函数可以有两个参数 eg： add

1.使用类
    定义一个add类 
    继承源代码 std::binary_function<int,int,void> 
    重载()
    #include <algorithom> \ <function>
    class add:public std::binary_function<int,int,void>
    {
        public:
            void operator()(int i,int j) const{
                cout << i + j << endl;
            } 
    };
    int main(){
        定义一个vector push_back几次
        std::for_each(v.begin,v.end(),std::bindlst(add(),10));
    }
    将v中的元素 每个都+10

使用std::for_each + boost::bind 简化以上代码 因为bind简化了不同函数之间的绑定
#include  <boost/bind.hpp> \ <algorithm>
    2.使用函数
        定义一个打印i+j的add函数
        int main(){
            //vector  push_back 3次
            std::for_each(v.begin(),v.end(),boost::bind(add,10,_1))
        }

        add需要两个参数 第一个参数是10 第二个参数是_1
        _1是占位符 还有_2 _3
        _1表示boost::bind()是一元函数 对应了for_each的第三个参数必须是一元函数
        _2表示二元

bind 例子
    std::for_each(v.begin(),v.end(),boost::bind(add,10,_1))
    std::sort(v.begin(),v.end(),boost::bind(compare,_1,_2))
    sort需要 两个参数的函数 作为第三个参数
    std::sort()进行容器排序 由于compare是二元函数 所以 sort(v.begin(),v.end(),compare)也可
    升序排列而不用修改compare()函数的定义
    boost::bind(compare,_2,_1) 改变排序的顺序


boost::ref() + boost::bind()  ---->>> bind()函数带有至少一个引用参数时 ref()用于处理引用
#include <boost/bind.hpp>
#include <algorithm>
//传递流对象 来打印信息
void add(int i,int j,ostream& o){ os << i+j << endl;}
int main(){
    //vector push_back
    std::for_each(v.begin(),v.end(),boost::bind(add,10,_1,boost::ref(cout)));
}

boost::bind 时值传递 传递std::cout会视图创建它的一份拷贝 std::cout不能直接使用

以引用方式传递常量对象： 使用模板boost::cref()


boost::function
#include <boost/function.hpp>
#include <cstdlib>
#include <cstring>

int main(){
    boost::function<int (const char*)> f = std::atoi;
    std::cout << f("1609") << endl;
    f = std::strlen;
    cout << f("1609") << endl;
}

可以定义之歌指针 指向具有特定签名的函数
定义了一个指针f 指向一个接受类型为const char* 返回值类型为int的函数
赋值给这个指针 

第一个将std::atoi()赋值给f 再将它重新赋值为std::strlen()

ps:给定的数据类型不需要精确匹配 strlen()的返回值是size_t类型 但是可以赋值给f

f是一个函数指针 被赋值的函数可以通过重载的operator()()操作符来调用
取决于当前被赋值的是哪一个函数

f若没有赋值就被调用 抛出 boost::bad_function_call异常：
    #include <boost/function.hpp> 
    #include <iostream> 

    int main() 
    { 
        try 
        { 
            boost::function<int (const char*)> f; 
            f(""); 
        } 
        catch (boost::bad_function_call &ex) 
        { 
            std::cout << ex.what() << std::endl; 
        } 
    } 

    要检查一个函数指针是否被赋值某个函数，可以使用 empty() 函数或 operator bool() 操作符


使用boost::function 类成员函数也可以被赋值给类型为boost::function的对象：

    #include<boost/function.hpp>
    struct world{
        void hello(ostream &os){
            os << "Hello World" << endl;
        }
    };
    int main(){
        boost::function<void (word*,ostream&)> f = &world::hello;
        world w;
        f(&w,boost::ref(cout));
    }

    function的第一个参数表示该函数 被调用的特定对象
    因此 必须是该特定类的指针
    后面的参数表示相应的成员函数的签名
    f使用了boost::ref() 进行传递引用

boost::Lambda 匿名函数
    #include <algorithm>
    //for_each打印 vector中的元素

    std::for_each(v.begin(),v.end(),print)
    改为lambda类型：
        std::for_each(v.begin(),v.end(),std::cout <<boost::lambda::_1 << "\n");

    lambda的占位符 _1 _2 _3
    与boost::bind不同：
        boost::lambda占位符有单独的名字空间
        通过使用占位符 容器v的元素可以通过 << 传给std::cout 以将它们写出到标准输出流
        注意 使用"\n" 代替 std::endl 
        一元endl模板函数要求的类型不同于lambda函数 cou << lambda::_1的函数

lambda将大于1的元素打印
    #include <boost/lambda/lambda.hpp>
    #include <boost/lambda/if.hpp>
    #include <algorithm>

    std::for_each(v.begin(),v.end(), boost::lambda::if_then(boost::lambda_1 > 1,cout << boost::lambda::_1 << "\n"))

boost/lambda/if.hpp 允许lambda再函数内部使用if语句
基本结构是 lambda::if_then() lambda::if_then_else() 和 lambda::if_then_else_return()

boost::signal
#include <boost/signal.hpp>
    signal 被实现为一个模板函数 具有被用作为事件处理器的函数的签名
    该签名是它的模板参数

    例子：
        #include <algorithm>
        #include <boost/signal.hpp>

        void func(){
            //cout 
        }
        int main(){
            boost::signal<void ()>s;
            s.connect(func);
            s();
        }
        //也可以使用function实现
        boost::function<void ()> f;
        f = func;
        f();
        

different：
    signal可以关联多个函数到特定信号
    boost::function 只能到单个函数

    signal关联多个：
        void f1(){}
        void f2(){}
        boost::signal<void ()>s;
        s.connect(f1);
        s.connect(f2);
        s();


signal执行的顺序：
    可以按照connect关联时的顺序
    可以重载来明确指定 重载添加一个int参数
    s.connect(1,f2);
    s.connect(0,f1);

signal释放给定信号关联：
    boost::signal<void ()>s;
    s.connect(f1);
    s.connect(f2);
    s.disconnect(f2);
    s();

signal 返回关联函数的数量:
    s.num_slots()
    s.empty()

释放所有关联：
    s.disconect_all_slots()

有返回值的signal 关联函数：
    boost::signal<int ()> s;
    只会返回之后一个关联的函数返回值

定制一个信号令每个返回值都相应被处理
使用合成器combiner作为第二个参数传递给 boost::signal

合成器类似于重载()操作符
这个操作符会被自动调用

signal 要求合成器定义一个 result_type类型 用于说明operator()()操作符返回值的类型
用于说明operator()()操作符返回值的类型 but cpp算法中缺少这个类型
所以在编译时会产生一个相应的错误

获取signal所有返回值：
    #include <boost/signal.hpp>
    #include <algorithm>

    int f1(){ return 1;}
    int f2(){ return 2;}

    int main(){
        boost::signal<int (),min_element<vector<int>> >s;
        s.connect(f1);
        s.connect(f2);
        vector<int> v = s();
        cout <<  << endl;
    }

    template <typename T>
    struct min_element{
        typedef T result_type;

        template <typename InputIterator>
        T operarot()(InputIterator first,InputIterator last)const{
            return T(first,last);
        }
    };

合成器重载了operator()()操作符的类 操作符会自动调用 传入两个迭代器 指向特定信号的所有返回值
将所有返回值保存在一个vector中 再由s()返回


连接 connections
函数可以通过connect和disconnect来管理
connect返回一个 boost::signals::connection的值

void func(){
    cout << endl;
}

int main(){
    boost::signal<void ()> s;
    boost::signals::connection c = s.connect(func);
    s();
    c.disconnect();
}

block阻塞
#include <boost/signal.hpp>

void f(){
    cout << endl;
}
int main(){
    boost::signal<void ()> s;
    boost::signal::connection c = s.connect(f);
    c.block();
    s();
    c.unblock();
    s();
}

自动释放连接
boost::signals::scoped_connection

int main(){
    boost::signal<void ()> s;
    {
        boost::signals::scoped_connection c = s.connect(f);
    }
    s(); //不会调用s 因为连接对象在信号触发之前被销毁
}

boost::bind关联一个对象到信息
一但对象被销毁，连接就自动释放

#include <boost/signal.hpp>
#include <boost/bind.hpp>

int main(){
    boost::signal<void ()> s;
    {
        std::auto_ptr<world> w(new world());
        s.connect(boost::bind(&world::hello,w.get()));
    }
    cout << s.num_slots() << endl;
    s();
}
class world:
public boost::signals::trackable{
    public:
        void hello() const{
            cout << ""hello world << endl;
        }
}

当使用对象的指针 而不是副本来关联函数至信号时
boost::signals::trackable可以显著简化连接的管理

字符串处理
区域设置参数：在标准cpp中封装了文化习俗相关的内容
locale

std::strcoll() 按照字典顺序比较第一个字符是否小于第二个
判断这两个字符中哪一个在字典中靠前

字符串算法库
boost::stringAlgorithms 很多字符串操作函数
字符串类型有 std::string std::wstring std::basic_string

将字符串转换为大写形式：
    boost::algorithm::to_upper_copy()
将字符串转换为小写形式:
    boost::algorithm::to_lower_copy()

字符串作为参数转换为大小写形式：
    boost::algorithm::to_upper()
    boost::algorithm::to_lower()

std::setlocale()为C函数进行区域设置的例子：

std::locale::global(std::locale("German")); 
std::string s = "Boris Schäling"; 
std::cout << boost::algorithm::to_upper_copy(s) << std::endl; 
std::cout << boost::algorithm::to_upper_copy(s, std::locale("German")) << std::endl; 



删除字符串中特定的某个字符：
    boost::algorithm::erase_all_copy()

删除字符串中首次出现的字符：
    boost::algorithm::erase_first_copy()

字符串头部或尾部删除：
    boost::algorithm::erase_head_copy()
    boost::algorithm::erase_tail_copy()


查找子串

std::locale::global(std::locale("German")); 
std::string s = "Boris Schäling"; 
boost::iterator_range<std::string::iterator> r = boost::algorithm::find_first(s, "Boris"); 
std::cout << r << std::endl; 
r = boost::algorithm::find_first(s, "xyz"); 
std::cout << r << std::endl;

在字符串中查找子串：
boost::algorithm::find_first()
boost::algorithm::find_last()
boost::algorithm::find_nth()
boost::algorithm::find_head()
boost::algorithm::find_tail()

共同点：
    返回类型为boost::iterator_range类的一对迭代器
    起源于 Boost.Range库
    因为操作符 << 由 boost::iterator_range类 重载而来
    单个搜索算法的结果可以直接写入标准输出流



连接两个字符串：
    boost::algorithm::join()
    接受一个字符串的容器 比如 vector<string>
    作为第一个参数
    根据第二个参数将这些字符串连接
    eg:
        boost::algorithm::join(v," ")

使用字符串替代子串的函数

    boost::algorithm::replace_first_copy()
    boost::algorithm::replace_nth_copy()boost::algorithm::replace_last_copy()boost::algorithm::replace_all_copy()boost::algorithm::replace_head_copy() boost::algorithm::replace_tail_copy()

    使用方法和查找和删除函数差不多

替代字符串的例子：
int main() 
{ 
  std::locale::global(std::locale("German")); 
  std::string s = "Boris Schäling"; 
  std::cout << boost::algorithm::replace_first_copy(s, "B", "D") << std::endl; 
  std::cout << boost::algorithm::replace_nth_copy(s, "B", 0, "D") << std::endl; 
  std::cout << boost::algorithm::replace_last_copy(s, "B", "D") << std::endl; 
  std::cout << boost::algorithm::replace_all_copy(s, "B", "D") << std::endl; 
  std::cout << boost::algorithm::replace_head_copy(s, 5, "Doris") << std::endl; 
  std::cout << boost::algorithm::replace_tail_copy(s, 8, "Becker") << std::endl; 
} 

修剪函数 自动去除字符串中的空格或者字符串的结束符
boost::algorithm::trim_left_copy_if()boost::algorithm::trim_right_copy_if() boost::algorithm::trim_copy_if()

修剪函数的例子：
#include <boost/algorithm/string.hpp> 
#include <locale> 
#include <iostream> 

int main() 
{ 
  std::locale::global(std::locale("German")); 
  std::string s = "--Boris Schäling--"; 
  std::cout << "." << boost::algorithm::trim_left_copy_if(s, boost::algorithm::is_any_of("-")) << "." << std::endl; 
  std::cout << "." <<boost::algorithm::trim_right_copy_if(s, boost::algorithm::is_any_of("-")) << "." << std::endl; 
  std::cout << "." <<boost::algorithm::trim_copy_if(s, boost::algorithm::is_any_of("-")) << "." << std::endl; 
} 


辅助函数 
boost::algorithm::is_any_of()
生成谓词以验证作为参数传入的字符是否在给定的字符串中存在


检查大小写
boost::algorithm::is_upper() 和 boost::algorithm::is_lower() 

比较两个字符串
boost::algorithm::starts_with()
boost::algorithm::ends_with()
boost::algorithm::contains()boost::algorithm::lexicographical_compare()

比较两个字符串实例

#include <boost/algorithm/string.hpp> 
#include <locale> 
#include <iostream> 

int main() 
{ 
  std::locale::global(std::locale("German")); 
  std::string s = "Boris Schäling"; 
  std::cout << boost::algorithm::starts_with(s, "Boris") << std::endl; 
  std::cout << boost::algorithm::ends_with(s, "Schäling") << std::endl; 
  std::cout << boost::algorithm::contains(s, "is") << std::endl; 
  std::cout << boost::algorithm::lexicographical_compare(s, "Boris") << std::endl; 
} 

字符串切割函数
将一个字符串拆分为一个字符串容器
boost::algorithm::split()
使用辅助函数boost::algorithm::is_space()创建一个谓词
在每个空格字符处分割字符串

字符串切割实例
和join()是反向
#include <boost/algorithm/string.hpp> 
#include <locale> 
#include <iostream> 
#include <vector> 

int main() 
{ 
  std::locale::global(std::locale("German")); 
  std::string s = "Boris Schäling"; 
  std::vector<std::string> v; 
  boost::algorithm::split(v, s, boost::algorithm::is_space()); 
  std::cout << v.size() << std::endl; 
} 

正则表达式
find_regex()搜索正则表达式

正则表达式 实例：

#include <boost/algorithm/string.hpp> 
#include <boost/algorithm/string/regex.hpp> 
#include <locale> 
#include <iostream> 

int main() 
{ 
  std::locale::global(std::locale("German")); 
  std::string s = "Boris Schäling"; 
  boost::iterator_range<std::string::iterator> r = boost::algorithm::find_regex(s, boost::regex("\\w\\s\\w")); 
  std::cout << r << std::endl; 
} 

正则表达式库
boost::regex 定义一个正则表达式
boost::smatch 保存搜索结果

#include <boost/regex.hpp>
#include <locale>
int main(){
  std::locale::global(std::locale("German")); 
  std::string s = "Boris Schäling"; 
  boost::regex expr("\\w+\\s\\w+"); 
  std::cout << boost::regex_match(s, expr) << std::endl;
}

boost::regex_match() 用于字符串与正则表达式的比较 
在整个字符串匹配正则表达式时其返回值为 true

搜索正则表达式
boost::regex_search()
#include <boost/regex.hpp> 
#include <locale> 
#include <iostream> 

int main() 
{ 
  std::locale::global(std::locale("German")); 
  std::string s = "Boris Schäling"; 
  boost::regex expr("(\\w+)\\s(\\w+)"); 
  boost::smatch what; 
  if (boost::regex_search(s, what, expr)) 
  { 
    std::cout << what[0] << std::endl; 
    std::cout << what[1] << " " << what[2] << std::endl; 
  } 
} 

boost::regex_search() 可以接受一个类型为 boost::smatch 的引用的参数用于储存结果